# Tryout

In [1]:
import torch
import torch.nn as nn

In [2]:
input_dim = 5
hidden_dim = 10
n_layers = 1

lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

In [3]:
# initialize hidden state of each cells
batch_size = 1
seq_len = 3

inp = torch.randn(batch_size, seq_len, input_dim)
hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)

print(inp.shape)
print(hidden_state.shape, cell_state.shape)

torch.Size([1, 3, 5])
torch.Size([1, 1, 10]) torch.Size([1, 1, 10])


In [4]:
# feed the input and hidden states and see what we’ll get back from it.
out, hidden = lstm_layer(inp, hidden)

print("Output shape: ", out.shape)
print("Hidden: ", hidden)

Output shape:  torch.Size([1, 3, 10])
Hidden:  (tensor([[[-8.4851e-02, -8.7748e-02,  6.2854e-02, -1.5089e-01,  7.9291e-05,
           1.6117e-01,  1.5947e-01,  1.4106e-01,  1.1534e-01,  1.3942e-01]]],
       grad_fn=<StackBackward>), tensor([[[-2.1368e-01, -1.8903e-01,  1.4674e-01, -3.4706e-01,  2.1826e-04,
           3.3946e-01,  3.7381e-01,  2.4257e-01,  3.2680e-01,  2.7158e-01]]],
       grad_fn=<StackBackward>))


In [5]:
# Obtaining the last output
out = out.squeeze()[-1, :]
print(out.shape)

torch.Size([10])


# Real implementation: Create PyTorch Dataset

In [1]:
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn

In [2]:
dir_path = "../phm_etching_01M01-02/M02_Groups_same_length/TotalFault_1_Flowcoolleak_Lot_runnum_9741-12620338.csv"

data_import = pd.read_csv(dir_path, encoding="utf8")
    
data_import.drop(columns=['time', 'Tool', 'Lot', 'TTF_FlowCool Pressure Dropped Below Limit',
       'TTF_Flowcool Pressure Too High Check Flowcool Pump', 'TTF_Flowcool leak', 'Lot-runnum'],
                     inplace=True)

print(data_import.to_numpy())

[[ 1.00000000e+00  1.26203380e+07  6.70000000e+01 ...  1.10558328e+00
   1.34003885e+00  2.54199820e+00]
 [ 1.00000000e+00  1.26203380e+07  6.70000000e+01 ...  1.10344075e+00
   1.33781144e+00  4.38892897e+00]
 [ 1.00000000e+00  1.26203380e+07  6.70000000e+01 ...  1.10408973e+00
   1.33848613e+00  3.85266160e+00]
 ...
 [ 1.00000000e+00  1.26203380e+07  6.70000000e+01 ...  1.10930283e+00
   1.34390580e+00 -7.64054656e-01]
 [ 1.00000000e+00  1.26203380e+07  6.70000000e+01 ...  1.10891630e+00
   1.34350395e+00 -3.83384113e-01]
 [ 1.00000000e+00  1.26203380e+07  6.70000000e+01 ...  1.10956530e+00
   1.34417866e+00 -9.91830116e-01]]


In [3]:
# preprocessing

# TotalFault_5_FlowCoolPressureDroppedBelowLimit-Flowcoolleak-FlowCoolPressureDroppedBelowLimit-FlowCoolPressureDroppedBelowLimit-FlowCoolPressureDroppedBelowLimit_Lot_runnum_9295-10815254

dir_path = "../phm_etching_01M01-02/M02_Groups_same_length/"
shuffle_indexes = [i for i in range(len(os.listdir(dir_path)))]
np.random.shuffle(shuffle_indexes)

data_x = np.zeros((len(os.listdir(dir_path)), 1005, 21))
data_y = np.zeros((len(os.listdir(dir_path)), 1005, 3))

# fault detection usage
# data_y = pd.DataFrame({
#     "Healthy": [0 for i in range(len(os.listdir(dir_path)))],
#     "FlowCoolPressureDroppedBelowLimit": [0 for i in range(len(os.listdir(dir_path)))],
#     "FlowcoolPressureTooHighCheckFlowcoolPump": [0 for i in range(len(os.listdir(dir_path)))],
#     "Flowcoolleak": [0 for i in range(len(os.listdir(dir_path)))]
# })

for fileIndex, file in enumerate(tqdm(os.listdir(dir_path))):
    
    fault_type_list = list(set(file.split("_")[2].split("-")))
    lot_runnum = file.split("_")[5].split(".")[0]
    
    data_import = pd.read_csv(dir_path + file, encoding="utf8")
    
    data_y[shuffle_indexes[fileIndex]] = data_import[['TTF_FlowCool Pressure Dropped Below Limit',
       'TTF_Flowcool Pressure Too High Check Flowcool Pump', 'TTF_Flowcool leak']].to_numpy()
    
    data_import.drop(columns=['time', 'Tool', 'Lot', 'TTF_FlowCool Pressure Dropped Below Limit',
       'TTF_Flowcool Pressure Too High Check Flowcool Pump', 'TTF_Flowcool leak', 'Lot-runnum'],
                     inplace=True)

     # rearrage value range into between -1 and 1
#     scaler = MinMaxScaler(feature_range=(-1, 1))
#     scaler.fit(data_import.to_numpy())
    
    data_x[shuffle_indexes[fileIndex]] = data_import.to_numpy()
    
    # fault_detection usage
#     for faultType in ["Healthy", "FlowCoolPressureDroppedBelowLimit",
#                       "FlowcoolPressureTooHighCheckFlowcoolPump", "Flowcoolleak"]:
#         if faultType in fault_type_list:
#             data_y.loc[shuffle_indexes[fileIndex], faultType] = 1
        
# fault detection usage
# data_y = data_y.to_numpy()
        
print(data_x.shape)
print(data_y.shape)

100%|██████████| 5013/5013 [00:39<00:00, 127.92it/s]

(5013, 1005, 21)
(5013, 1005, 3)


In [5]:
# split for training, validation, testing
training_ratio = 0.7
validation_ratio = 0.2
testing_ratio = 0.1

train_indice = int(np.around(len(data_x) * training_ratio, 0))
validation_indice = int(np.around(len(data_x) * training_ratio, 0)) +\
int(np.around(len(data_x) * validation_ratio, 0))

x_train = data_x[:train_indice]
y_train = data_y[:train_indice]
x_validation = data_x[train_indice:validation_indice]
y_validation = data_y[train_indice:validation_indice]
x_test = data_x[validation_indice:]
y_test = data_y[validation_indice:]

print(x_train.shape, x_validation.shape, x_test.shape)
print(y_train.shape, y_validation.shape, y_test.shape)

(3509, 1005, 21) (1003, 1005, 21) (501, 1005, 21)
(3509, 1005, 3) (1003, 1005, 3) (501, 1005, 3)


# Activate CUDA

In [6]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()
print(is_cuda)

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

True


# LSTM Modeling

#### code reference: 
  * https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=vIWvJCpOVmwU
  * https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

In [31]:
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import torch
import torch.nn as nn

In [35]:
# create tensor dataset by pytorch
data_x = Variable(torch.Tensor(data_x))
data_y = Variable(torch.Tensor(data_y))

trainX = Variable(torch.Tensor(x_train))
trainY = Variable(torch.Tensor(y_train))

validX = Variable(torch.Tensor(x_validation))
validY = Variable(torch.Tensor(y_validation))

testX = Variable(torch.Tensor(x_test))
testY = Variable(torch.Tensor(y_test))

In [36]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [37]:
num_epochs = 2000
learning_rate = 0.01

input_size = 21
hidden_size = 2
num_layers = 1

num_classes = 3

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

c:\users\user\anaconda3\envs\nlp\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([3509, 1005, 3])) that is different to the input size (torch.Size([3509, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (3509) must match the size of tensor b (1005) at non-singleton dimension 1

In [19]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, seq_length, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length
        
        self.lstm_model = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                           batch_first=True)
        
        
    def forward(self, x_input):
        '''
        Inputs: input, (h_0, c_0)
        
        h_0 of shape (num_layers * num_directions, batch, hidden_size): 
        tensor containing the initial hidden state for each element in the batch. 
        If the LSTM is bidirectional, num_directions should be 2, else it should be 1. 
        If proj_size > 0 was specified, the shape has to be (num_layers * num_directions, 
        batch, proj_size).
        
        c_0 of shape (num_layers * num_directions, batch, hidden_size): 
        tensor containing the initial cell state for each element in the batch.
        '''
        h_0 = Variable(torch.zeros(self.num_layers, x_input.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(self.num_layers, x_input.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        '''
        Outputs: output, (h_n, c_n)
        
        output of shape (seq_len, batch, num_directions * hidden_size): 
        tensor containing the output features (h_t) from the last layer of the LSTM, for each t. 
        If a torch.nn.utils.rnn.PackedSequence has been given as the input, 
        the output will also be a packed sequence. If proj_size > 0 was specified, 
        output shape will be (seq_len, batch, num_directions * proj_size).
        For the unpacked case, the directions can be separated using 
        output.view(seq_len, batch, num_directions, hidden_size), 
        with forward and backward being direction 0 and 1 respectively. 
        Similarly, the directions can be separated in the packed case.
        
        h_n of shape (num_layers * num_directions, batch, hidden_size): 
        tensor containing the hidden state for t = seq_len. 
        If proj_size > 0 was specified, h_n shape will be 
        (num_layers * num_directions, batch, proj_size).
        Like output, the layers can be separated using 
        h_n.view(num_layers, num_directions, batch, hidden_size) and similarly for c_n.
        
        c_n of shape (num_layers * num_directions, batch, hidden_size): 
        tensor containing the cell state for t = seq_len.
        '''
        ula, (h_out, _) = self.lstm_model(x_input, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out
        

In [26]:
# model configurations
num_epochs = 2000
learning_rate = 0.01

input_size = 21
seq_length = 1005
hidden_size = 2
num_layers = 1
num_classes = 3

lstm = LSTM(num_classes=num_classes, input_size=input_size, hidden_size=hidden_size,
           num_layers=num_layers, seq_length=seq_length)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# train model
for epoch in range(num_epochs):
    outputs = lstm(x_train)
    
    # initialize optimizer
    optimizer.zero_grad()
    
    # obtain loss and do back propagation
    loss = criterion(outputs, y_train)
    loss.backward()
    
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch:", str(epoch), "Loss:", loss.item())
    

TypeError: 'int' object is not callable